**General Instructions:**

- Collaborations between students during problem-solving phase on a discussion basis is OK
- However: individual code programming and submissions per student are required
- Code sharing is strictly prohibited
- We will run checks for shared code, general plagiarism and AI-generated solutions
- Any fraud attempt will lead to an auto fail of the entire course
- Do not use any additional packages except for those provided in the task templates
- Please use Julia Version 1.10.x to ensure compatibility
- Please only write between the `#--- YOUR CODE STARTS HERE ---#` and `#--- YOUR CODE ENDS HERE ---#` comments
- Please do not delete, add any cells or overwrite cells other than the solution cells (**Tip:** If you use a jupyerhub IDE, you should not be able to add or delete cells and write in the non-solution cells by default)

# Exercise Sheet 09

## Recap

### First-Order Necessecary Conditions

The overall goal of this lecture series is to get a
feeling for solving the general nonlinear problem
$$
\begin{aligned}
&\min_{\mathbf w\in ℝ^q}
  L(\mathbf w)
      &\text{subject to}\\
&c_i(\mathbf w) = 0, &∀i\in \mathcal E,\\
&c_i(\mathbf w) ≤ 0, &∀i\in \mathcal I,
\end{aligned}
\tag{NL}
$$
where $\mathcal E\subset ℕ_0$ and $\mathcal I \subset ℕ_0$ index
the nonlinear equality and inequality constraints.
The so called **Lagrangian** $\mathcal L$ of this problem is
$$
\mathcal L(\mathbf w, \mathbf λ, \mathbf μ)
=
L(\mathbf w) + \sum_{i \in \mathcal I} λ_i c_i(\mathbf w) + \sum_{i \in \mathcal E} μ_i c_i(\mathbf w).
$$
Under certain conditions, the Lagrangian can be used to give
necessary conditions for a point $\mathbf w^*$ to be optimal for
(NL).
If $\mathbf w^*$ is locally optimal, then there are multipliers $\mathbf λ^*$
and $\mathbf μ^*$ such that
$$
\begin{aligned}
∇_{\mathbf w} \mathcal L(\mathbf w^*, \mathbf λ^*, \mathbf μ^*) &= \mathbf 0,  \\
c_i(\mathbf w^*) &= 0, & ∀i \in \mathcal E, \\
c_i(\mathbf w^*) &≤ 0, & ∀i \in \mathcal I, \\
λ_i &\geq 0, & ∀i\in \mathcal I,\\
λ_i c_i(\mathbf w^*) &= 0 & ∀i\in \mathcal I.\\
\end{aligned}
$$
These conditions are stated as Theorem 12.1 in “Numerical Optimization” by Nocedal,
and are referred to as the KKT conditions.
The last line, the complementary slackness condition,
tells us, that the multipliers of the inactive
inequality constraints ($c_i(\mathbf w^*) < 0$) are zero.
The first line can be re-written as
$$
\nabla_{\mathbf w}
\mathcal L(\mathbf w^*, \mathbf λ^*, \mathbf μ^*)
=
∇L(\mathbf w)
+
\sum_{i \in \mathcal A(\mathbf w^*)}
λ_i \nabla c_i(\mathbf w^*)
+
\sum_{i\in \mathcal E}
μ_i \nabla c_i(\mathbf w^*)
=
\mathbf 0,
$$
with the active constraint indices $\mathcal A (\mathbf w^*) = \{i \in \mathcal I: c_i(\mathbf w^*) = 0\}$.

## Active-Set Method for Convex QP

An important building block for solving (NL) with
**sequential quadratic programming** (SQP) is the
minimization of the convex quadratic problem
$$
\begin{aligned}
&\min_{\mathbf p \in ℝ^q}
  \frac{1}{2} \mathbf p^T \mathbf H \mathbf p
+
  \mathbf g^T \mathbf p,
&\text{subject to}\\
&\mathbf c_i^T \mathbf p + d_i = 0,  &∀i \in \mathcal E, \\
&\mathbf c_i^T \mathbf p + d_i ≤ 0,  &∀i \in \mathcal I,
\end{aligned}
\tag{QP}
$$
where $\mathbf H$ is a square $q\times q$ matrix and positive semi-definite,
and $\mathbf g$ is a vector in $ℝ^q$.
$\mathcal E\subset ℕ_0$ is the index set for equality constraints,
$\mathcal I\subset ℕ_0$ is the index set for inequality constraints,
and $\mathbf c_i, i\in \mathcal I\cup \mathcal E,$ are vectors in $ℝ^q$.

To solve (NL) using SQP methods, we take an initial guess $\mathbf w^{(0)}$ and solve problems of the form (QP) to
obtain a step $\mathbf p^{(\ell)}$, resulting in updates $\mathbf w^{(\ell+1)} = \mathbf w^{(\ell)} + \mathbf p^{(\ell)}$.
The matrix $\mathbf H$ is the Hessian of $\mathcal L(\mathbf w^{(\ell)})$ and $\mathbf g$ is the loss gradient at $\mathbf w^{(\ell)}$.

But for now, we focus on just solving (QP).
Best forget about the outer iteration indices $(\ell)$ …
Given a concrete instance of (QP), we want to find optimal $\mathbf p^*$ using the Active-Set method.
The Active-Set method is an iterative scheme in which we start with an initial guess
$\mathbf p^{(0)} \in ℝ^q$
and again solve a sequence of sub-problems to determine some step
$\mathbf s^{(k)} = \mathbf p^{(k+1)} - \mathbf p^{(k)}$.
Actually, we have a two-stage procedure.
First, we obtain $\tilde {\mathbf s}^{(k)}$ as the optimizer of
$$
\begin{aligned}
&\min_{\mathbf s \in ℝ^q}
\frac{1}{2} \mathbf s^T \mathbf H \mathbf s
+ \mathbf s^T \mathbf h^{(k)}
&\text{s.t.}\\
&\mathbf c_i^T \mathbf s = 0,  &∀i \in \mathcal E, \\
&\mathbf c_i^T \mathbf s = 0,  &∀i \in \mathcal A^{(k)}.
\end{aligned}
\tag{SP}
$$
Here, we have defined
$$
\mathbf h^{(k)} = \mathbf H \mathbf p^{(k)} + \mathbf g.
$$
The index set $\mathcal A^{(k)} \subset \mathcal I$ is the
iteration-dependent *working set* of active constraints at $\mathbf p^{(k)}$:
$$
\mathcal A^{(k)} = \left\{
   i \in \mathcal I: \mathbf c_i^T \mathbf p^{(k)} + d_i = 0
\right\}.
$$
In a second step, we compute a step-size (Exercise 1c) to obtain update step $\mathbf s^{(k)}$ from $\tilde{\mathbf s}^{(k)}$.


### Exercise 1a)  (1 point)
In the cell below, complete the code to implement a function, that identifies the active
indices of the linear inequality constraints.
(That is, the inequality value is between 0 and `tol`.)
Suppose, the linear inequality constraints are given in matrix form by a matrix `C_ineq`
and `d_ineq`, such that
$$
\mathbf C_{\text{ineq}} \mathbf p^{(k)} + \mathbf d_{\text{ineq}} \stackrel{!}{≤} \mathbf 0,
$$
i.e., the rows of `C_ineq` hold the constraint vectors, and we are interested in
a subset of the row indices. `active_indices` should return a `Vector{Int}` holding these indices.

In [ ]:
function active_indices(C_ineq, d_ineq, pk, pthrough=nothing; tol=1e-10)
    # ignore the `pthrough` argument. it is used for grading.
    
    @assert size(C_ineq, 1) == length(d_ineq)

    # 1) Initialize empty `Int` vector to hold active indices.
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    
    # 2) Evaluate constraint equation C * p + d
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    
    # 3) Iterate over result vector entries
    #    a) if a value is > `tol`, throw an error, e.g., `error("pk is not feasible")`
    #    b) if *absolute* value of entry is <= `tol`, add index to index array
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#

    # 4) return index array
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
end

In [ ]:
# This cell defines a patched version of `active_indices`. Don't delete it!

When checking `active_indices`, we can use the `Test` library to test for exceptions:

In [ ]:
import Test
let
    # w1 ≤ 0.5*w2
    # w1 ≤ 1
    C_ineq = [
        1 -0.5;
        1 0
    ]
    d_ineq = [0, -1]
    pk = [0.5, 1]
    Ak = active_indices(C_ineq, d_ineq, pk)
    @assert Ak isa AbstractVector{<:Integer}
    @assert only(Ak) == 1

    pk = [2, 1]
    Test.@test_throws Exception active_indices(C_ineq, d_ineq, pk)
end

### Exercise 1b) (1 point)
In the cell below, complete the code to implement a function, that solves the KKT
equations for the subproblem (SP),
$$
\begin{bmatrix}
\mathbf H & \mathbf C_{\mathcal A^{(k)}}^T & \mathbf C_{\text{eq}}^T \\
\mathbf C_{\mathcal A^{(k)}}& \mathbf 0 & \mathbf 0 \\
\mathbf C_{\text{eq}} & \mathbf 0 & \mathbf 0
\end{bmatrix}
\begin{bmatrix}
\tilde{\mathbf s}^{(k)} \\
\mathbf λ^{(k)} \\
\mathbf μ^{(k)} \\
\end{bmatrix}
\stackrel{!}{=}
\begin{bmatrix}
-\mathbf h^{(k)} \\
\mathbf 0\\
\mathbf 0
\end{bmatrix}.
$$
These equations result from writing down the KKT conditions for (SP).

In [ ]:
function solve_step_problem(H, C_Ak, C_eq, hk, pthrough=nothing)
    # ignore the `pthrough` argument. it is used for grading.

    num_vars = size(H, 1)  # length of sk_tilde
    @assert size(H, 2) == num_vars
    @assert size(C_Ak, 2) == num_vars
    @assert size(C_eq, 2) == num_vars
    @assert length(hk) == num_vars

    num_ineq = size(C_Ak, 1) # length of lambdak
    num_eq = size(C_eq, 1)   # length of muk

    # change `KKT` to hold the correct matrix
    KKT = rand(num_vars + num_ineq + num_eq, num_vars + num_ineq + num_eq)
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    
    RHS = zeros(num_vars + num_ineq + num_eq)
    # change `RHS` to hold the correct right-hand-side matrix
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#

    # solve the linear equation system and set `sk`, `lambdak` and `muk`
    # to hold the multipliers
    sk_tilde = lambdak = muk = nothing
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    return sk_tilde, lambdak, muk
end

In [ ]:
# This cell defines a patched version of `solve_step_problem`. Don't delete it!

Below are the tests for `solve_step_problem`. We use `LinearAlgebra` to initialize `H` as the identity matrix.

In [ ]:
import LinearAlgebra as LA

In [ ]:
let
    H = 1.0 .* LA.I(2)
    C_Ak = [1 -0.5]
    C_eq = [1 0]
    pk = [1, 2]
    hk = H * pk
    sk_tilde, λk, μk = solve_step_problem(H, C_Ak, C_eq, hk)

    @assert sk_tilde isa AbstractVector
    @assert λk isa AbstractVector
    @assert μk isa AbstractVector
    @assert length(sk_tilde) == 2
    @assert length(λk) == 1
    @assert length(μk) == 1
    # `sk_tilde` should be close to zero:
    @assert sum(sk_tilde .^ 2) <= 1e-10
    nothing
end

### Exercise 1c) (1.5 points)
Once $\tilde{\mathbf s}^{(k)}$ is available from `solve_step_problem`,
we need to determine a step size $α_k \in [0, 1]$.
$α_k$ should be as large as possible, and chosen so
that $\mathbf p^{(k+1)} = \mathbf p^{(k)} + α_k \tilde{\mathbf s}^{(k)}$ is
feasible for **all** the linear constraints of (QP).
The vector $\mathbf p^{(k+1)}$ is feasible for any $α_k$ and
all the constraints indexed by $\mathcal E$
and $\mathcal A^{(k)}$ by virtue of the KKT conditions of (SP).
Suppose $i\in \mathcal I\smallsetminus A^{(k)}$.
If $\mathbf c_i^T \tilde{\mathbf s}^{(k)} \leq 0$, then for all $α_k \geq 0$
we have
$$
\mathbf c_i^T(\mathbf p^{(k)} + α_k \tilde{\mathbf s}^{(k)}) + d_i
\leq
\mathbf c_i^T \mathbf p^{(k)} + d_i
\leq 0.
$$
Thus, we only have to consider the case
$i \in \mathcal I\smallsetminus \mathcal A^{(k)}$
and $\mathbf c_i^T \tilde{\mathbf s}^{(k)} > 0$.
In that case, we solve
$$
α_k
\leq
\frac{-d_i - \mathbf c_i^T \mathbf p^{(k)}}{\mathbf c_i^T \tilde{\mathbf s}^{(k)}}.
$$
Implement `stepsize_and_blocking_index`
to return the stepsize $α_k$ and a blocking index.
Use this formula:
$$
α_k = \min
\left\{
1,
\min_{
    \substack{
        i \in \mathcal I\smallsetminus \mathcal A^{(k)},\\
        \mathbf c_i^T \tilde{\mathbf s}^{(k)} > 0
    }
}
\frac{-d_i - \mathbf c_i^T \mathbf p^{(k)}}{\mathbf c_i^T \tilde{\mathbf s}^{(k)}}
\right\}.
$$
A "blocking index" is an index $i\in\mathcal I\smallsetminus \mathcal A^{(k)}$ such that the expression
of the inner $\min$ operator is strictly smaller than $1$.
If there is no blocking index, return the stepsize and $-1$.

In [ ]:
function stepsize_and_blocking_index(C_ineq, d_ineq, pk, sk_tilde, Ak, pthrough=nothing)
    # ignore the `pthrough` argument. it is used for grading.

    alphak = 1.0
    j = -1

    # determine correct stepsize `alphak` according to the formula above;
    # `sk_tilde` is the unscaled step vector, `pk` are the variables,
    # the rows of `C_ineq` hold the vectors $c_i$,
    # and the vector `d_ineq` holds the linear constraint offsets:
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    return alphak, j
end

In [ ]:
# This cell defines a patched version of `stepsize_and_blocking_index`. Don't delete it!

Let's test our function:

In [ ]:
let
    C_ineq = [1 -0.5]
    d_ineq = [0,]

    pk = [1.0, 2.0]
    sk_tilde = [0.0; -2.0]
    Ak = Int[]
    # Check if the returned values make sense:
    αk, j = stepsize_and_blocking_index(C_ineq, d_ineq, pk, sk_tilde, Ak)
    @assert αk isa Real
    @assert j isa Integer
    @assert -1e-10 <= αk <= 1 + 1e-10
    @assert j >= -1

    # Now, we should be able to take a full step:
    Ak = [1,]
    sk = zeros(2)

    αk, j = stepsize_and_blocking_index(C_ineq, d_ineq, pk, sk_tilde, Ak)
    @assert αk ≈ 1
    @assert j == -1
end

### Exercise 1d) (1.5 + 0.5 points)
We now have all building blocks for the Active-Set Method.
The algorithm stops, if a step $\mathbf s^{(k)}$ is zero
and all multipliers $λ_i^{(k)}$ are non-negative.
Otherwise, the working set is augmented.
Likewise, the working set is augmented in case of a non-zero
step with $α_k < 1$.
Complete the marked sections in the code below:

In [ ]:
function solve_convex_qp(
    H, g, C_eq, d_eq, C_ineq, d_ineq, p0, pthrough=nothing;
    tol=1e-10, max_iter=100
)
    # ignore the `pthrough` argument. it is used for grading.

    
    pk = copy(p0)
    # 1) check feasibilty of p0 with respect to equality constraints:
    #    compute vector `r_eq` holding the equality constraint values and
    #    check if all entries have absolute values <= tol; elsewise `error`.
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    
    # 2) determine active indices for `pk` with `active_indices` and store in vector `Ak`
    # IMPORTANT: Pass `pthrough` to `active_indices`!
    Ak = Int[]
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#

    k = 1
    while k <= max_iter
        # Compute the current objective value `Lk` of (QP) at `pk`
        Lk = Inf
        #--- YOUR CODE STARTS HERE ---#
        
        #--- YOUR CODE ENDS HERE ---#

        @info "Iteration $k, L(pk)=$(Lk)."

        # extract active inequality constraint matrix
        C_Ak = @view(C_ineq[Ak, :])

        # solve SP to get step:
        # first, compute `hk` and then call `solve_step_problem`;
        # pass the correct constraint matrices and
        # store results in `sk_tilde, λk, μk`.
        # IMPORTANT: Pass `pthrough` to `solve_step_problem`!
        hk = sk_tilde = λk = μk = nothing
        #--- YOUR CODE STARTS HERE ---#
        
        #--- YOUR CODE ENDS HERE ---#

        if sqrt(sum(sk_tilde .^ 2)) <= tol
            @info "Zero step."
            # if `sk_tilde` is (nearly) zero, then see if all multipliers are non-negative
            problem_solved = isempty(λk)
            if !problem_solved
                λ_min, λ_min_index = findmin(λk)
                problem_solved = λ_min >= -tol
            end
            if problem_solved
                @info "Problem solved."
                break
            end
            # if there are negative multipliers,
            # remove the index corresponding to smallest index for next iteration
            deleteat!(Ak, λ_min_index)
        else
            # `sk_tilde` is different from zero, compute stepsize `αk` and see if there
            # is a blocking index `j`.
            # If there is a blocking index, add it to `Ak` for the next iteration.
            # IMPORTANT: Pass `pthrough` to `stepsize_and_blocking_index`!
            #--- YOUR CODE STARTS HERE ---#
            
            #--- YOUR CODE ENDS HERE ---#
            # Also, update `pk` for the next iteration:
            #--- YOUR CODE STARTS HERE ---#
            
            #--- YOUR CODE ENDS HERE ---#
        end
        k += 1
    end
    return pk, k
end

Here is how we can test your implementation:

In [ ]:
pthrough=nothing
H = 1.0 .* LA.I(2)
g = zeros(2)

C_ineq = [1 -0.5]
d_ineq = [0,]

C_eq = [1 0]
d_eq = [-1,]

p0 = [1, 4]
popt, k = solve_convex_qp(H, g, C_eq, d_eq, C_ineq, d_ineq, p0, pthrough; max_iter=10)
@assert popt isa Vector
@assert length(popt) == 2
@assert k == 2
# check that solution satisfies constraints
@assert abs(popt[1] - 1) <= 1e-9
@assert abs(popt[1] - 0.5 * popt[2]) <= 1e-9
# check, if optimal value is reduced
L(p) = 0.5 * p'H * p + g'p
L0 = L(p0)
Lopt = L(popt)
@assert Lopt < L0

If any of the tests failed, it could be due to an error in any of the functions
`active_indices`, `solve_step_problem`, or `stepsize_and_blocking_index`.
We provide patched implementations, enabled by the `pthrough` argument, so that the routine should work in the 
graded tests.
So even if the cell below fails for you, it might run on the grading server.

In [ ]:
# NBGrader does not like logging, so we disable it some times:
import Logging

In [ ]:
let pthrough = Val(true);
    Logging.with_logger(Logging.NullLogger()) do
        H = 1.0 .* LA.I(2)
        g = zeros(2)

        C_ineq = [1 -0.5]
        d_ineq = [0,]

        C_eq = [1 0]
        d_eq = [-1,]

        p0 = [1, 4]
        popt, k = solve_convex_qp(H, g, C_eq, d_eq, C_ineq, d_ineq, p0, pthrough; max_iter=10)
        @assert popt isa Vector
        @assert length(popt) == 2
        @assert k == 2
        # check that solution satisfies constraints
        @assert abs(popt[1] - 1) <= 1e-9
        @assert abs(popt[1] - 0.5 * popt[2]) <= 1e-9
        # check, if optimal value is reduced
        L(p) = 0.5 * p'H * p + g'p
        L0 = L(p0)
        Lopt = L(popt)
        @assert Lopt < L0

    end
end

In the cell below, we can see if you successfully disallow infeasible points:

In [ ]:
let
    # same setup as before:
    H = 1.0 .* LA.I(2)
    g = zeros(2)
    C_ineq = [1 -0.5]
    d_ineq = [0,]
    C_eq = [1 0]
    d_eq = [-1,]

    p0 = [2, 1]

    Test.@test_throws Exception solve_convex_qp(H, g, C_eq, d_eq, C_ineq, d_ineq, p0; max_iter=10)
end

## JuMP for Linear and Quadratic Programming

The [`JuMP` package](https://jump.dev/JuMP.jl/stable/) provides a unified syntax to
define problems with linear or quadratic constraints.
It also supports automatic re-formulation of problems to suit the needs of
the various solvers that are supported.
[The list of solvers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers) is
pretty extensive, and the ability to switch between them so easily is one of the
main reasons why `JuMP` is very popular with practitioners in the fields of optimization
and optimal control.

If we were to write an actual SQP algorithm, and did not want to bother with
the Active-Set-Method subroutine ourselves, we could use `JuMP` to solve the sub-problems.
In exercise 2a) you are going to use `JuMP` and the solver `COSMO` to solve the following
problem:
$$
\begin{alignedat}{3}
&\min_{\mathbf p\in ℝ^2}
   (p_1 - 1)^2 + (p_2 - 2.5)^2
&& &&\text{subject to}\\
&-p_1 + 2 p_2 - 2 &&≤ 0,\\
&+p_1 + 2 p_2 - 6 &&≤ 0,\\
&+p_1 - 2 p_2 - 2 &&≤ 0,\\
&+p_1 && \geq 0,\\
&+p_2 && \geq 0.\\
\end{alignedat}
\tag{P}
$$

![problem plot](fig_cosmo.png)

### Exercise 2a) (0.5 + 0.5 + 1 points)
Below is a function to solve the problem (P) using `JuMP`.
However, there are 5 mistakes in the code.
Fix them. If unsure, consult the [Tutorial.](https://jump.dev/JuMP.jl/stable/tutorials/getting_started/getting_started_with_JuMP/#Getting-started-with-JuMP)

In [ ]:
import JuMP
import JuMP: @variable, @objective, @constraint, optimize!, Model, value
import COSMO

In [ ]:
p_cosmo = model = nothing
let
    # we are using a local scope here to be extra careful, and to avoid poluting the
    # global scope
    global p_cosmo, model
    model = Model(COSMO.Optimizer)
    @variable(model, p[1:3] .>= -1)

    @objective(model, Min, (p[1] - 1)^2 + (p[2] - 2.5)^2 + p[3])

    @constraint(model, -p[1] + 2 * p[2] + 2 <= 0)
    @constraint(model, p[1] + 2 * p[2] - 6 <= 0)
    @constraint(model, +p[1] - 2 * p[2] - 2 <= 2)
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#

    optimize!(model)

    p_cosmo = value.(p)
    nothing
end

Check success:

In [ ]:
@assert JuMP.termination_status(model) == JuMP.OPTIMAL

Check dimensions and constraint types:

In [ ]:
@assert length(p_cosmo) == 2
@assert JuMP.num_constraints(model; count_variable_in_set_constraints=true) == 5

Check optimal value:

In [ ]:
let p = p_cosmo
    @assert all(p .>= 0)
    @assert -p[1] + 2 * p[2] - 2 <= 1e-10
    @assert p[1] + 2 * p[2] - 6 <= 1e-10
    @assert +p[1] - 2 * p[2] - 2 <= 1e-10
end

![problem plot](fig_cosmo2.png)

### Exercise 2b) (1 + 1 points)
As indicated, it is possible to use `JuMP` in many different ways.
We could also provide the affine-linear inequality constraints with matrices.
Set the 3 x 2 matrix `A_cosmo` and the 3-vector `b_cosmo`, so that `A_cosmo * x .<= b_cosmo`
describes the constraints of (P) (other than the non-negativity constraints.)

In [ ]:
A_cosmo = zeros(3, 2)
b_cosmo = zeros(3)
#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

As a quick test, we can check the previous solution:

In [ ]:
@assert all(A_cosmo * p_cosmo .<= b_cosmo .+ 1e-10)

Moreover, we can also rewrite the objective function as
`0.5 * p' * H_cosmo * p + p'g_cosmo + c_cosmo`,
with a 2 x 2 matrix `H_cosmo` and vector `g_cosmo` and constant `c_cosmo`:

In [ ]:
H_cosmo = zeros(2, 2)
g_cosmo = zeros(2)
c_cosmo = 0
#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

Check against last optimal value:

In [ ]:
@assert JuMP.objective_value(model) ≈ 0.5 * p_cosmo'H_cosmo * p_cosmo + p_cosmo'g_cosmo + c_cosmo

Here is how to use matrices with `JuMP`:

In [ ]:
let
    global H_cosmo, g_cosmo, c_cosmo
    global A_cosmo, b_cosmo, p_cosmo
    model = Model(COSMO.Optimizer)
    @variable(model, p[1:2] .>= 0)
    @objective(model, Min, 0.5 * p'H_cosmo * p + p'g_cosmo + c_cosmo)
    @constraint(model, A_cosmo * p .<= b_cosmo)
    optimize!(model)

    p_opt = value.(p)
    @assert p_opt ≈ p_cosmo
end

### Exercise 2c) (0.5 points)
Incidently, we have restated our problem in a form nearly suitable for our Active-Set solver
from exercise 1.
Augment `A_cosmo` and `b_cosmo` by including the non-negativity constraints and
name the resulting matrix `C_ineq_as`, and the vector `b_ineq_as`:

In [ ]:
C_ineq_as = zeros(5, 2)
b_ineq_as = zeros(5)
#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

Let's again check with the last solution:

In [ ]:
@assert all(C_ineq_as * p_cosmo .<= b_ineq_as .+ 1e-10 )

Finally, our solver should return the same optimum, hopefully:

In [ ]:
let H = H_cosmo, g = g_cosmo, C_ineq = C_ineq_as, d_ineq = -b_ineq_as
    global p_cosmo
    C_eq = zeros(0, 2)
    d_eq = []
    p0 = ones(2)
    p_qp = p0
    p_qp, _ = solve_convex_qp(H, g, C_eq, d_eq, C_ineq, d_ineq, p0)

    @assert p_cosmo ≈ p_qp
end